# ETL: Inversión

## Librerias

In [48]:
# Generales
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Funciones Snowflake
import funciones as snow_func

In [49]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [50]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [51]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }

# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x000001644FD31E50>}


In [52]:
# Crear objeto de conexión
conn = session.connection

## 1. Bases de inversión: Años cerrados y corridos juntos

In [53]:
# Solo se debe cambiar la ubicación del archivo
path_ied = "C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/INVERSION/"
ied_pais_file = '2024-II Banrep - IED por país.xlsx'
ice_pais_file = '2024-II Banrep - ICE por país.xlsx'
ied_actividad_file = '2024-II Banrep - IED por actividad.xlsx'
ice_actividad_file = '2024-II Banrep - ICE por actividad.xlsx'

#### 1.1. IED País

In [54]:
# Leer el archivo
df_ied_corrido_pais = pd.read_excel(path_ied + ied_pais_file, sheet_name='base_TI')
# Columnas para realizar transformar en long
id_vars = ['PAIS']
value_vars = df_ied_corrido_pais.columns[1:]
# Pasar a long y agregar tipo de inversión
df_ied_corrido_pais = df_ied_corrido_pais.melt(id_vars=id_vars, value_vars=value_vars, var_name='Año', value_name='Valor')
df_ied_corrido_pais['Inversión'] = 'IED'

#### 1.2. ICE País

In [55]:
# Leer el archivo
df_ice_corrido_pais = pd.read_excel(path_ied + ice_pais_file, sheet_name='base_TI')
# Columnas para realizar transformar en long
id_vars = ['PAIS']
value_vars = df_ice_corrido_pais.columns[1:]
# Pasar a long y agregar tipo de inversión
df_ice_corrido_pais = df_ice_corrido_pais.melt(id_vars=id_vars, value_vars=value_vars, var_name='Año', value_name='Valor')
df_ice_corrido_pais['Inversión'] = 'ICE'

#### 1.3. IED e ICE por país

In [56]:
# Unir ambos tipos de inversión
df_ied_ice_corrido_pais = pd.concat([df_ied_corrido_pais, df_ice_corrido_pais])
# Eliminar espacios y total
df_ied_ice_corrido_pais['Año'] = df_ied_ice_corrido_pais['Año'].str.replace("Total", "")
df_ied_ice_corrido_pais['Año'] = df_ied_ice_corrido_pais['Año'].str.replace(" ", "")
# Nuevos nombres de columnas (No pueden empezar con números ni tener * o tildes O Ñ)
df_ied_ice_corrido_pais = df_ied_ice_corrido_pais.rename(columns={
    'PAIS': 'PAIS',
    'Año': 'YEAR',
    'Valor': 'VALOR',	
    'Inversión': 'TIPO'
})

# Reemplazar 'Inglaterra' y 'Escocia' por 'Reino Unido' en la columna 'PAIS'
df_ied_ice_corrido_pais['PAIS'] = df_ied_ice_corrido_pais['PAIS'].replace({
    'INGLATERRA': 'REINO UNIDO',
    'ESCOCIA': 'REINO UNIDO'
})

#### 1.4. IED actividad

In [57]:
# Leer el archivo
df_ied_corrido_actividad = pd.read_excel(path_ied + ied_actividad_file, sheet_name='base_Tableau')
df_ied_corrido_actividad = df_ied_corrido_actividad[(~df_ied_corrido_actividad['Actividad económica'].isin(['Total actividades económicas', 'Subtotal Resto de Sectores']))]
# Columnas para realizar transformar en long
id_vars = ['Actividad económica']
value_vars = df_ied_corrido_actividad.columns[1:]
# Pasar a long y agregar tipo de inversión
df_ied_corrido_actividad = df_ied_corrido_actividad.melt(id_vars=id_vars, value_vars=value_vars, var_name='Año', value_name='Valor')
df_ied_corrido_actividad['Inversión'] = 'IED'

#### 1.5. ICE actividad

In [58]:
# Leer el archivo
df_ice_corrido_actividad = pd.read_excel(path_ied + ice_actividad_file, sheet_name='base_TI')
# Columnas para realizar transformar en long
id_vars = ['Actividad económica']
value_vars = df_ice_corrido_actividad.columns[1:]
# Pasar a long y agregar tipo de inversión
df_ice_corrido_actividad = df_ice_corrido_actividad.melt(id_vars=id_vars, value_vars=value_vars, var_name='Año', value_name='Valor')
df_ice_corrido_actividad['Inversión'] = 'ICE'

#### 1.6. IED e ICE por actividad

In [59]:
# Unir ambos tipos de inversión
df_ied_ice_corrido_actividad = pd.concat([df_ied_corrido_actividad, df_ice_corrido_actividad])
# Eliminar espacios y total
df_ied_ice_corrido_actividad['Año'] = df_ied_ice_corrido_actividad['Año'].str.replace("Total", "")
df_ied_ice_corrido_actividad['Año'] = df_ied_ice_corrido_actividad['Año'].str.replace(" ", "")
df_ied_ice_corrido_actividad['Actividad económica'] = df_ied_ice_corrido_actividad['Actividad económica'].str.replace(" ³", "")
# Nuevos nombres de columnas (No pueden empezar con números ni tener * o tildes O Ñ)
df_ied_ice_corrido_actividad = df_ied_ice_corrido_actividad.rename(columns={
    'Actividad económica': 'ACTIVIDAD',
    'Año': 'YEAR',
    'Valor': 'VALOR',	
    'Inversión': 'TIPO'
})

## 2. Parámetros (cambiar el mes)

In [60]:
# Crear un DataFrame que represente TEMP_PARAMETROS_CERRADO
TEMP_PARAMETROS_CERRADO = pd.DataFrame({
    'T_1_YEAR': ['2022'],
    'T_YEAR': ['2023']
})

# Mostrar el DataFrame resultante
TEMP_PARAMETROS_CERRADO

,T_1_YEAR,T_YEAR
0,2022,2023


In [61]:
# Crear un DataFrame que represente TEMP_PARAMETROS_CORRIDO
TEMP_PARAMETROS_CORRIDO = pd.DataFrame({
    'T_1_YEAR': ['2023-2'],
    'T_YEAR': ['2024-2']
})

# Mostrar el DataFrame resultante
TEMP_PARAMETROS_CORRIDO

,T_1_YEAR,T_YEAR
0,2023-2,2024-2


In [62]:
# Crear un número que contenga el número de trimestre corrido de interés
trimestre_corrido = 2

In [63]:
# Prepocesar datos de ied e ice corrido de país (para mostrar el acumulado del año)

# Filtrar años cerrados
df_ied_ice_pais_corrido_insumo = df_ied_ice_corrido_pais[df_ied_ice_corrido_pais['YEAR'].str.contains('-')]

# Dividir la columna YEAR en dos nuevas columnas: AÑO y TRIMESTRE
df_ied_ice_pais_corrido_insumo['AÑO'] = df_ied_ice_pais_corrido_insumo['YEAR'].str.split('-').str[0]
df_ied_ice_pais_corrido_insumo['TRIMESTRE'] = df_ied_ice_pais_corrido_insumo['YEAR'].str.split('-').str[1].astype(int)

# Seleccionar trimestre igual al elegido o menor
df_ied_ice_pais_corrido_insumo = df_ied_ice_pais_corrido_insumo[df_ied_ice_pais_corrido_insumo['TRIMESTRE'] <= trimestre_corrido]

# Agrupar por PAIS, AÑO, TIPO y sumar VALOR
df_ied_ice_pais_corrido_insumo = df_ied_ice_pais_corrido_insumo.groupby(['PAIS', 'AÑO', 'TIPO'], as_index=False)['VALOR'].sum()

# Convertir el AÑO y TRIMESTRE seleccionados de nuevo al formato original "YEAR"
df_ied_ice_pais_corrido_insumo['YEAR'] = df_ied_ice_pais_corrido_insumo['AÑO'] + f'-{trimestre_corrido}'

# Eliminar la columna 'AÑO'
df_ied_ice_pais_corrido_insumo = df_ied_ice_pais_corrido_insumo.drop(columns=['AÑO'])

# Reordenar las columnas para tener 'YEAR' en el lugar correcto
df_ied_ice_pais_corrido_insumo = df_ied_ice_pais_corrido_insumo[['PAIS', 'YEAR', 'VALOR', 'TIPO']]

In [64]:
df_ied_ice_pais_corrido_insumo

,PAIS,YEAR,VALOR,TIPO
0,AFGANISTAN,2012-2,0.0000000000,IED
1,AFGANISTAN,2013-2,0.0000000000,IED
2,AFGANISTAN,2014-2,0.0000000000,IED
3,AFGANISTAN,2015-2,0.0000000000,IED
4,AFGANISTAN,2016-2,0.0000000000,IED
...,...,...,...,...
1763,VENEZUELA,2022-2,1.0776693759,IED
1764,VENEZUELA,2023-2,2.4060131700,ICE
1765,VENEZUELA,2023-2,2.0648098326,IED
1766,VENEZUELA,2024-2,-0.5092812200,ICE


In [65]:
# Prepocesar datos de ied e ice corrido de actividad (para mostrar el acumulado del año)

# Filtrar años cerrados
df_ied_ice_corrido_actividad_insumo = df_ied_ice_corrido_actividad[df_ied_ice_corrido_actividad['YEAR'].str.contains('-')]

# Dividir la columna YEAR en dos nuevas columnas: AÑO y TRIMESTRE
df_ied_ice_corrido_actividad_insumo['AÑO'] = df_ied_ice_corrido_actividad_insumo['YEAR'].str.split('-').str[0]
df_ied_ice_corrido_actividad_insumo['TRIMESTRE'] = df_ied_ice_corrido_actividad_insumo['YEAR'].str.split('-').str[1].astype(int)

# Seleccionar trimestre igual al elegido o menor
df_ied_ice_corrido_actividad_insumo = df_ied_ice_corrido_actividad_insumo[df_ied_ice_corrido_actividad_insumo['TRIMESTRE'] <= trimestre_corrido]

# Agrupar por ACTIVIDAD, AÑO, TIPO y sumar VALOR
df_ied_ice_corrido_actividad_insumo = df_ied_ice_corrido_actividad_insumo.groupby(['ACTIVIDAD', 'AÑO', 'TIPO'], as_index=False)['VALOR'].sum()

# Convertir el AÑO y TRIMESTRE seleccionados de nuevo al formato original "YEAR"
df_ied_ice_corrido_actividad_insumo['YEAR'] = df_ied_ice_corrido_actividad_insumo['AÑO'] + f'-{trimestre_corrido}'

# Eliminar la columna 'AÑO'
df_ied_ice_corrido_actividad_insumo = df_ied_ice_corrido_actividad_insumo.drop(columns=['AÑO'])

# Reordenar las columnas para tener 'YEAR' en el lugar correcto
df_ied_ice_corrido_actividad_insumo = df_ied_ice_corrido_actividad_insumo[['ACTIVIDAD', 'YEAR', 'VALOR', 'TIPO']]

## 2.5 Verificar países

In [66]:
# Se trae la tabla completa de geografía para el conteo de empresas
query_geografia = """
SELECT *
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL;
"""
# Ejecutar
df_geo = snow_func.snowflake_sql(conn, query_geografia)

In [67]:
# Elegir columnas de interés en cada caso

# Países
df_geo = df_geo[['PAIS_INVERSION_BANREP', 'COUNTRY_OR_AREA']]

# Inversión
df_paises_inversion = df_ied_ice_corrido_pais[['PAIS']].drop_duplicates()

In [68]:
# Realizar unión
df_paises_inversion = df_paises_inversion.merge(df_geo, how='left', left_on='PAIS', right_on='PAIS_INVERSION_BANREP')

# No deben salir países de verdad nulos, si salen, se deben agregar al modelo relacional
# La única excepción es PAISES BAJOS que viene sin datos del Banrep
df_paises_inversion[df_paises_inversion['COUNTRY_OR_AREA'].isna()]

,PAIS,PAIS_INVERSION_BANREP,COUNTRY_OR_AREA
80,PAISES BAJOS,NaN,NaN


## 3. Crear totales de control

#### Funciones

In [69]:
def generar_totales(df, df_parametros, valor_col):
    """
    Genera un DataFrame con los totales agrupados para los años especificados en df_parametros.
    
    Parámetros:
    df (DataFrame): DataFrame original que contiene los datos a agrupar.
    df_parametros (DataFrame): DataFrame que contiene los años 'T_1_YEAR' y 'T_YEAR'.
    valor_col (str): Nombre de la columna que contiene los valores que se van a sumar.
    
    Retorna:
    DataFrame con los totales sumados.
    """
    # Sumar los valores para el año T_1_YEAR
    suma_t_1 = df[df['YEAR'] == df_parametros['T_1_YEAR'].iloc[0]][valor_col].sum()
    
    # Sumar los valores para el año T_YEAR
    suma_t = df[df['YEAR'] == df_parametros['T_YEAR'].iloc[0]][valor_col].sum()
    
    # Crear el DataFrame con los resultados
    df_total_usd = pd.DataFrame({
        'TABLA' : ['TOTAL'],
        'CATEGORIA': ['TOTAL'],
        'SUMA_USD_T_1': [suma_t_1],
        'SUMA_USD_T': [suma_t]
    })

    # Crear sumas de control
    suma_t_1_control =  df_total_usd['SUMA_USD_T_1']
    suma_t_control =  df_total_usd['SUMA_USD_T']
      
    return suma_t_1_control, suma_t_control

### Totales de control

In [70]:
# Estos totales se calculan con la misma lógica al interior de las funciones de la siguiente sección para realizar la verificación.

# TOTALES IED Y ICE
# IED
# Cerrado
suma_total_ied_t_1_control_cerrado, suma_total_ied_t_control_cerrado = generar_totales(df_ied_ice_corrido_pais[df_ied_ice_corrido_pais['TIPO']=='IED'], TEMP_PARAMETROS_CERRADO, 'VALOR')
# Corrido
suma_total_ied_t_1_control_corrido, suma_total_ied_t_control_corrido = generar_totales(df_ied_ice_pais_corrido_insumo[df_ied_ice_pais_corrido_insumo['TIPO']=='IED'], TEMP_PARAMETROS_CORRIDO, 'VALOR')

# ICE
# Cerrado
suma_total_ice_t_1_control_cerrado, suma_total_ice_t_control_cerrado = generar_totales(df_ied_ice_corrido_pais[df_ied_ice_corrido_pais['TIPO']=='ICE'], TEMP_PARAMETROS_CERRADO, 'VALOR')
# Corrido
suma_total_ice_t_1_control_corrido, suma_total_ice_t_control_corrido = generar_totales(df_ied_ice_pais_corrido_insumo[df_ied_ice_pais_corrido_insumo['TIPO']=='ICE'], TEMP_PARAMETROS_CORRIDO, 'VALOR')

In [71]:
# Verificar totales
texto = f"""
Total ied cerrado T-1 y T: {suma_total_ied_t_1_control_cerrado.iloc[0]} y {suma_total_ied_t_control_cerrado.iloc[0]}
Total ied corrido T-1 y T: {suma_total_ied_t_1_control_corrido.iloc[0]} y {suma_total_ied_t_control_corrido.iloc[0]}

Total ice cerrado T-1 y T: {suma_total_ice_t_1_control_cerrado.iloc[0]} y {suma_total_ice_t_control_cerrado.iloc[0]}
Total ice corrido T-1 y T: {suma_total_ice_t_1_control_corrido.iloc[0]} y {suma_total_ice_t_control_corrido.iloc[0]}

"""
print(texto)


Total ied cerrado T-1 y T: 17182.521349905943 y 17144.522051986492
Total ied corrido T-1 y T: 9417.0104136117 y 6719.620596224162

Total ice cerrado T-1 y T: 3383.5048732111113 y 1174.8141488064514
Total ice corrido T-1 y T: 416.9588856770601 y 2142.097940045353




## 4. Crear agrupaciones

#### Funciones

In [72]:
def generar_tabla_agrupada(df, agrupacion, unidad, tabla, categoria_col, valor_col, temp_parametros):
    """
    Genera una tabla agregada con métricas financieras de un DataFrame de inversión, agrupando por diferentes categorías.
    
    Optimizada para reducir el tiempo de ejecución filtrando previamente y usando operaciones vectorizadas.
    """
    
    # Filtrar los datos por los dos años antes de agrupar para evitar filtrado en cada agregación
    df_t1 = df[df['YEAR'] == temp_parametros['T_1_YEAR'].iloc[0]]
    df_t = df[df['YEAR'] == temp_parametros['T_YEAR'].iloc[0]]

    # Determinar las columnas de agrupación
    if categoria_col == unidad:
        group_cols = [unidad]
    else:
        group_cols = [categoria_col] + ([unidad] if unidad in df.columns else [])
    
    # Agrupar para cada año por separado
    suma_t1 = df_t1.groupby(group_cols)[valor_col].sum().reset_index(name='SUMA_USD_T_1')
    suma_t = df_t.groupby(group_cols)[valor_col].sum().reset_index(name='SUMA_USD_T')

    # Unir los resultados de los dos años
    df_agrupado = pd.merge(suma_t1, suma_t, on=group_cols, how='outer')#.fillna(0)

    # Agregar las columnas 'AGRUPACION' y 'TABLA'
    df_agrupado['AGRUPACION'] = agrupacion
    df_agrupado['TABLA'] = tabla

    # Agregar la columna 'UNIDAD'
    if unidad in df.columns:
        df_agrupado['UNIDAD'] = df_agrupado[unidad]
    else:
        df_agrupado['UNIDAD'] = unidad

    # Asignar la columna 'CATEGORIA'
    if categoria_col == unidad:
        df_agrupado['CATEGORIA'] = df_agrupado['UNIDAD']
    else:
        df_agrupado = df_agrupado.rename(columns={categoria_col: 'CATEGORIA'})

    # Calcular las diferencias absoluta y porcentual utilizando operaciones vectorizadas
    df_agrupado['DIFERENCIA_ABSOLUTA_T'] = df_agrupado['SUMA_USD_T'] - df_agrupado['SUMA_USD_T_1']
    df_agrupado['DIFERENCIA_PORCENTUAL_T'] = np.where(
        df_agrupado['SUMA_USD_T_1'] == 0,
        np.where(df_agrupado['SUMA_USD_T'] > 0, 100, np.where(df_agrupado['SUMA_USD_T'] == 0, 0, -100)),
        (df_agrupado['DIFERENCIA_ABSOLUTA_T'] / df_agrupado['SUMA_USD_T_1']) * 100
    )

    # Reordenar las columnas
    df_agrupado = df_agrupado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T', 'DIFERENCIA_ABSOLUTA_T', 'DIFERENCIA_PORCENTUAL_T']]

    return df_agrupado

In [73]:
def generar_tablas_paises(df, temp_parametros):
    """
    Esta función genera una tabla agregada por país y realiza validaciones de los totales de inversión extranjera directa (IED) 
    e inversión colombiana en el exterior (ICE). Calcula las sumas para dos años y compara los resultados con los totales esperados.

    Parámetros:
    df (pd.DataFrame): El DataFrame que contiene los datos de entrada, como inversiones por país, tipo (IED o ICE), y valores.
    temp_parametros (pd.DataFrame): El DataFrame que contiene los parámetros temporales con los años de comparación (T_1_YEAR y T_YEAR).

    Retorna:
    tuple: Un DataFrame con los resultados agregados por país y un DataFrame con los resultados de validación.
    """

    # Crear agrupación por país, agrupando los datos por el campo 'PAIS' y calculando las sumas por TIPO ('IED' o 'ICE').
    df_paises = generar_tabla_agrupada(df=df, agrupacion='PAISES', unidad='PAIS', tabla='INVERSIÓN PAÍSES', 
                                       categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)

    # Filtrar las observaciones que tengan valores diferentes de cero en las columnas 'SUMA_USD_T_1' o 'SUMA_USD_T'.
    df_paises = df_paises[(df_paises['SUMA_USD_T_1'] != 0) | (df_paises['SUMA_USD_T'] != 0)]

    # Crear la tabla de totales sumando todas las inversiones de IED e ICE, agrupadas bajo 'TOTAL'.
    df_totales = generar_tabla_agrupada(df=df, agrupacion='PAISES', unidad='TOTAL', tabla='INVERSIÓN PAÍSES', 
                                        categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)

    # Concatenar los resultados de los países con los totales.
    ST_PAISES = pd.concat([df_paises, df_totales])

    # Cambiar los nombres de las columnas para reflejar las sumas de inversión en lugar de las sumas de USD.
    ST_PAISES.rename(columns={'SUMA_USD_T_1': 'SUMA_INVERSION_T_1', 'SUMA_USD_T': 'SUMA_INVERSION_T'}, inplace=True)

    # Verificar la calidad de los datos para IED e ICE calculando los totales y comparándolos con los totales de control.

    # Generar los totales de control para IED.
    suma_total_ied_t_1_control, suma_total_ied_t_control = generar_totales(df[df['TIPO'] == 'IED'], temp_parametros, 'VALOR')

    # Generar los totales de control para ICE.
    suma_total_ice_t_1_control, suma_total_ice_t_control = generar_totales(df[df['TIPO'] == 'ICE'], temp_parametros, 'VALOR')

    # Definir un margen de tolerancia para la comparación de los totales (en este caso, 1).
    tolerancia = 1

    # --- Validación para IED ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a IED.
    df_ied = df_paises[df_paises['CATEGORIA'] == 'IED']

    # Comparar la suma de T_1_YEAR para IED.
    suma_parcial_ied_t_1 = df_ied['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED.
    suma_parcial_ied_t = df_ied['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para IED en un DataFrame.
    df_resultados_ied = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación para ICE ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a ICE.
    df_ice = df_paises[df_paises['CATEGORIA'] == 'ICE']

    # Comparar la suma de T_1_YEAR para ICE.
    suma_parcial_ice_t_1 = df_ice['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE.
    suma_parcial_ice_t = df_ice['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para ICE en un DataFrame.
    df_resultados_ice = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # --- Validación de los totales para IED ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a IED.
    df_ied_total = df_totales[(df_totales['CATEGORIA'] == 'IED') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para IED en los totales.
    suma_parcial_ied_t_1 = df_ied_total['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED en los totales.
    suma_parcial_ied_t = df_ied_total['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para IED en un DataFrame.
    df_resultados_ied_total = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación de los totales para ICE ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a ICE.
    df_ice_totales = df_totales[(df_totales['CATEGORIA'] == 'ICE') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para ICE en los totales.
    suma_parcial_ice_t_1 = df_ice_totales['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE en los totales.
    suma_parcial_ice_t = df_ice_totales['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para ICE en un DataFrame.
    df_resultados_ice_total = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # Concatenar todos los resultados de validación (IED, ICE, y sus totales).
    df_resultados_validacion = pd.concat([df_resultados_ied, df_resultados_ice, df_resultados_ied_total, df_resultados_ice_total], ignore_index=True)

    # Devolver los resultados finales: la tabla de países y el DataFrame de validación.
    return ST_PAISES, df_resultados_validacion


In [74]:
def generar_tablas_actividades(df, temp_parametros):
    """
    Genera una tabla de inversión agrupada por actividades económicas, valida los totales para Inversión Extranjera Directa (IED) e 
    Inversión Colombiana en el Exterior (ICE), y compara los resultados con los totales de control.

    Parámetros:
    ----------
    df (pd.DataFrame): DataFrame con los datos de inversión, que contiene columnas como 'TIPO' (IED o ICE), 'VALOR', y 'ACTIVIDAD'.
    temp_parametros (pd.DataFrame): DataFrame con los parámetros temporales, incluyendo las columnas 'T_1_YEAR' y 'T_YEAR' para 
                                    realizar las comparaciones entre dos años.

    Retorna:
    --------
    tuple: 
        - ST_ACTIVIDADES (pd.DataFrame): Tabla de inversión por actividades económicas, con sumas y diferencias de inversión.
        - df_resultados_validacion (pd.DataFrame): DataFrame que contiene los resultados de validación para las sumas parciales 
                                                   y totales de IED e ICE.
    """

    # Generar la tabla de actividades agrupada por actividad económica
    df_actividades = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad, agrupacion='ACTIVIDADES', unidad='ACTIVIDAD', 
                                            tabla='INVERSIÓN ACTIVIDADES', categoria_col='TIPO', valor_col='VALOR', 
                                            temp_parametros=temp_parametros)

    # Eliminar las filas donde las sumas para ambos años sean cero
    df_actividades = df_actividades[(df_actividades['SUMA_USD_T_1'] != 0) | (df_actividades['SUMA_USD_T'] != 0)]

    # Generar la tabla de totales para todas las actividades, agrupadas bajo 'TOTAL'
    df_totales = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad, agrupacion='ACTIVIDADES', unidad='TOTAL', 
                                        tabla='INVERSIÓN ACTIVIDADES', categoria_col='TIPO', valor_col='VALOR', 
                                        temp_parametros=temp_parametros)

    # Concatenar los resultados de las actividades económicas con los totales
    ST_ACTIVIDADES = pd.concat([df_actividades, df_totales])

    # Cambiar los nombres de las columnas para reflejar las sumas de inversión en lugar de USD
    ST_ACTIVIDADES.rename(columns={'SUMA_USD_T_1': 'SUMA_INVERSION_T_1', 'SUMA_USD_T': 'SUMA_INVERSION_T'}, inplace=True)

    # Verificar la calidad de los datos para IED e ICE comparando con los totales de control

    # Generar los totales de control para IED
    suma_total_ied_t_1_control, suma_total_ied_t_control = generar_totales(df[df['TIPO'] == 'IED'], temp_parametros, 'VALOR')

    # Generar los totales de control para ICE
    suma_total_ice_t_1_control, suma_total_ice_t_control = generar_totales(df[df['TIPO'] == 'ICE'], temp_parametros, 'VALOR')

    # Definir el margen de tolerancia para la comparación de los totales
    tolerancia = 1

    # --- Validación para IED ---
    # Filtrar el DataFrame de actividades para obtener los registros correspondientes a IED
    df_ied = df_actividades[df_actividades['CATEGORIA'] == 'IED']

    # Comparar la suma de T_1_YEAR para IED
    suma_parcial_ied_t_1 = df_ied['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED
    suma_parcial_ied_t = df_ied['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para IED en un DataFrame
    df_resultados_ied = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación para ICE ---
    # Filtrar el DataFrame de actividades para obtener los registros correspondientes a ICE
    df_ice = df_actividades[df_actividades['CATEGORIA'] == 'ICE']

    # Comparar la suma de T_1_YEAR para ICE
    suma_parcial_ice_t_1 = df_ice['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE
    suma_parcial_ice_t = df_ice['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para ICE en un DataFrame
    df_resultados_ice = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # --- Validación de los totales para IED ---
    # Filtrar el DataFrame de totales para obtener los registros correspondientes a IED
    df_ied_total = df_totales[(df_totales['CATEGORIA'] == 'IED') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para IED en los totales
    suma_parcial_ied_t_1 = df_ied_total['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED en los totales
    suma_parcial_ied_t = df_ied_total['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para IED en un DataFrame
    df_resultados_ied_total = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación de los totales para ICE ---
    # Filtrar el DataFrame de totales para obtener los registros correspondientes a ICE
    df_ice_totales = df_totales[(df_totales['CATEGORIA'] == 'ICE') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para ICE en los totales
    suma_parcial_ice_t_1 = df_ice_totales['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE en los totales
    suma_parcial_ice_t = df_ice_totales['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para ICE en un DataFrame
    df_resultados_ice_total = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # Concatenar todos los resultados de validación (IED, ICE, y sus totales)
    df_resultados_validacion = pd.concat([df_resultados_ied, df_resultados_ice, df_resultados_ied_total, df_resultados_ice_total], ignore_index=True)

    # Devolver los resultados finales: la tabla de actividades y el DataFrame de validación
    return ST_ACTIVIDADES, df_resultados_validacion

In [75]:
def generar_tablas_actividades_cerrado(df, temp_parametros):
    temp_parametros = TEMP_PARAMETROS_CERRADO
    df_actividades = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad, agrupacion='ACTIVIDADES', unidad='ACTIVIDAD', tabla='INVERSIÓN ACTIVIDADES', 
                                        categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)
    df_actividades = df_actividades[(df_actividades['SUMA_USD_T_1'] != 0) | (df_actividades['SUMA_USD_T'] != 0)]
    df_totales = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad, agrupacion='ACTIVIDADES', unidad='TOTAL', tabla='INVERSIÓN ACTIVIDADES', 
                                        categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)
    
    # Concatenar los resultados de las actividades con los totales.
    ST_ACTIVIDADES = pd.concat([df_actividades, df_totales])

    # Cambiar los nombres de las columnas para reflejar las sumas de inversión en lugar de las sumas de USD.
    ST_ACTIVIDADES.rename(columns={'SUMA_USD_T_1': 'SUMA_INVERSION_T_1', 'SUMA_USD_T': 'SUMA_INVERSION_T'}, inplace=True)
    
    # Verificar la calidad de los datos para IED e ICE calculando los totales y comparándolos con los totales de control.

    # Generar los totales de control para IED.
    suma_total_ied_t_1_control, suma_total_ied_t_control = generar_totales(df[df['TIPO'] == 'IED'], temp_parametros, 'VALOR')

    # Generar los totales de control para ICE.
    suma_total_ice_t_1_control, suma_total_ice_t_control = generar_totales(df[df['TIPO'] == 'ICE'], temp_parametros, 'VALOR')

    # Definir un margen de tolerancia para la comparación de los totales (en este caso, 1).
    tolerancia = 1

    # --- Validación para IED ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a IED.
    df_ied = df_actividades[df_actividades['CATEGORIA'] == 'IED']

    # Comparar la suma de T_1_YEAR para IED.
    suma_parcial_ied_t_1 = df_ied['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED.
    suma_parcial_ied_t = df_ied['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para IED en un DataFrame.
    df_resultados_ied = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación para ICE ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a ICE.
    df_ice = df_actividades[df_actividades['CATEGORIA'] == 'ICE']

    # Comparar la suma de T_1_YEAR para ICE.
    suma_parcial_ice_t_1 = df_ice['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE.
    suma_parcial_ice_t = df_ice['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para ICE en un DataFrame.
    df_resultados_ice = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # --- Validación de los totales para IED ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a IED.
    df_ied_total = df_totales[(df_totales['CATEGORIA'] == 'IED') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para IED en los totales.
    suma_parcial_ied_t_1 = df_ied_total['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED en los totales.
    suma_parcial_ied_t = df_ied_total['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para IED en un DataFrame.
    df_resultados_ied_total = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación de los totales para ICE ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a ICE.
    df_ice_totales = df_totales[(df_totales['CATEGORIA'] == 'ICE') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para ICE en los totales.
    suma_parcial_ice_t_1 = df_ice_totales['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE en los totales.
    suma_parcial_ice_t = df_ice_totales['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para ICE en un DataFrame.
    df_resultados_ice_total = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # Concatenar todos los resultados de validación (IED, ICE, y sus totales).
    df_resultados_validacion = pd.concat([df_resultados_ied, df_resultados_ice, df_resultados_ied_total, df_resultados_ice_total], ignore_index=True)

    # Devolver los resultados finales: la tabla de países y el DataFrame de validación.
    return ST_ACTIVIDADES, df_resultados_validacion



In [76]:
def generar_tablas_actividades_corrido(df, temp_parametros):
    temp_parametros = TEMP_PARAMETROS_CORRIDO
    df_actividades = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad_insumo, agrupacion='ACTIVIDADES', unidad='ACTIVIDAD', tabla='INVERSIÓN ACTIVIDADES', 
                                        categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)
    df_actividades = df_actividades[(df_actividades['SUMA_USD_T_1'] != 0) | (df_actividades['SUMA_USD_T'] != 0)]
    df_totales = generar_tabla_agrupada(df=df_ied_ice_corrido_actividad_insumo, agrupacion='ACTIVIDADES', unidad='TOTAL', tabla='INVERSIÓN ACTIVIDADES', 
                                        categoria_col='TIPO', valor_col='VALOR', temp_parametros=temp_parametros)
    
    # Concatenar los resultados de las actividades con los totales.
    ST_ACTIVIDADES = pd.concat([df_actividades, df_totales])

    # Cambiar los nombres de las columnas para reflejar las sumas de inversión en lugar de las sumas de USD.
    ST_ACTIVIDADES.rename(columns={'SUMA_USD_T_1': 'SUMA_INVERSION_T_1', 'SUMA_USD_T': 'SUMA_INVERSION_T'}, inplace=True)
    
    # Verificar la calidad de los datos para IED e ICE calculando los totales y comparándolos con los totales de control.

    # Generar los totales de control para IED.
    suma_total_ied_t_1_control, suma_total_ied_t_control = generar_totales(df[df['TIPO'] == 'IED'], temp_parametros, 'VALOR')

    # Generar los totales de control para ICE.
    suma_total_ice_t_1_control, suma_total_ice_t_control = generar_totales(df[df['TIPO'] == 'ICE'], temp_parametros, 'VALOR')

    # Definir un margen de tolerancia para la comparación de los totales (en este caso, 1).
    tolerancia = 1

    # --- Validación para IED ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a IED.
    df_ied = df_actividades[df_actividades['CATEGORIA'] == 'IED']

    # Comparar la suma de T_1_YEAR para IED.
    suma_parcial_ied_t_1 = df_ied['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED.
    suma_parcial_ied_t = df_ied['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para IED en un DataFrame.
    df_resultados_ied = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación para ICE ---

    # Filtrar el DataFrame para obtener únicamente los registros correspondientes a ICE.
    df_ice = df_actividades[df_actividades['CATEGORIA'] == 'ICE']

    # Comparar la suma de T_1_YEAR para ICE.
    suma_parcial_ice_t_1 = df_ice['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE.
    suma_parcial_ice_t = df_ice['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación para ICE en un DataFrame.
    df_resultados_ice = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # --- Validación de los totales para IED ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a IED.
    df_ied_total = df_totales[(df_totales['CATEGORIA'] == 'IED') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para IED en los totales.
    suma_parcial_ied_t_1 = df_ied_total['SUMA_USD_T_1'].sum()
    resultado_ied_t_1 = abs(round(suma_parcial_ied_t_1) - round(suma_total_ied_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para IED en los totales.
    suma_parcial_ied_t = df_ied_total['SUMA_USD_T'].sum()
    resultado_ied_t = abs(round(suma_parcial_ied_t) - round(suma_total_ied_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para IED en un DataFrame.
    df_resultados_ied_total = pd.DataFrame({
        'AGRUPACION': ['IED'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['IED'],
        'CATEGORIA': ['IED'],
        'SUMA_USD_T_1': [resultado_ied_t_1],
        'SUMA_USD_T': [resultado_ied_t]
    })

    # --- Validación de los totales para ICE ---

    # Filtrar el DataFrame de totales para obtener únicamente los registros correspondientes a ICE.
    df_ice_totales = df_totales[(df_totales['CATEGORIA'] == 'ICE') & (df_totales['UNIDAD'] == 'TOTAL')]

    # Comparar la suma de T_1_YEAR para ICE en los totales.
    suma_parcial_ice_t_1 = df_ice_totales['SUMA_USD_T_1'].sum()
    resultado_ice_t_1 = abs(round(suma_parcial_ice_t_1) - round(suma_total_ice_t_1_control.iloc[0])) <= tolerancia

    # Comparar la suma de T_YEAR para ICE en los totales.
    suma_parcial_ice_t = df_ice_totales['SUMA_USD_T'].sum()
    resultado_ice_t = abs(round(suma_parcial_ice_t) - round(suma_total_ice_t_control.iloc[0])) <= tolerancia

    # Guardar los resultados de validación de totales para ICE en un DataFrame.
    df_resultados_ice_total = pd.DataFrame({
        'AGRUPACION': ['ICE'],
        'UNIDAD': ['TOTAL'],
        'TABLA': ['ICE'],
        'CATEGORIA': ['ICE'],
        'SUMA_USD_T_1': [resultado_ice_t_1],
        'SUMA_USD_T': [resultado_ice_t]
    })

    # Concatenar todos los resultados de validación (IED, ICE, y sus totales).
    df_resultados_validacion = pd.concat([df_resultados_ied, df_resultados_ice, df_resultados_ied_total, df_resultados_ice_total], ignore_index=True)

    # Devolver los resultados finales: la tabla de países y el DataFrame de validación.
    return ST_ACTIVIDADES, df_resultados_validacion


### 4.1. ST_PAISES_CERRADO

In [77]:
# Crear df y validador
ST_PAISES_CERRADO, df_validador = generar_tablas_paises(df= df_ied_ice_corrido_pais, temp_parametros = TEMP_PARAMETROS_CERRADO)

In [78]:
# Todo debe estar en true para no tener errores
df_validador

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T
0,IED,TOTAL,IED,IED,True,True
1,ICE,TOTAL,ICE,ICE,True,True
2,IED,TOTAL,IED,IED,True,True
3,ICE,TOTAL,ICE,ICE,True,True


### 4.2. ST_PAISES_CORRIDO

In [79]:
# Crear df y validador
ST_PAISES_CORRIDO, df_validador = generar_tablas_paises(df= df_ied_ice_pais_corrido_insumo, temp_parametros = TEMP_PARAMETROS_CORRIDO)

In [80]:
# Todo debe estar en true para no tener errores
df_validador

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T
0,IED,TOTAL,IED,IED,True,True
1,ICE,TOTAL,ICE,ICE,True,True
2,IED,TOTAL,IED,IED,True,True
3,ICE,TOTAL,ICE,ICE,True,True


In [81]:
# Cambio a nombres de versión original (asegura compatibilidad con el código anterior )
ST_PAISES_CORRIDO = ST_PAISES_CORRIDO.rename(columns={'DIFERENCIA_ABSOLUTA_T': 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL_T' : 'DIFERENCIA_PORCENTUAL'})

### 4.3. ST_ACTIVIDADES_CERRADO

In [82]:
# Crear df y validador
ST_ACTIVIDADES_CERRADO, df_validador = generar_tablas_actividades_cerrado(df=df_ied_ice_corrido_actividad, temp_parametros=TEMP_PARAMETROS_CERRADO)

In [83]:
# Todo debe estar en true para no tener errores
df_validador

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T
0,IED,TOTAL,IED,IED,True,True
1,ICE,TOTAL,ICE,ICE,True,True
2,IED,TOTAL,IED,IED,True,True
3,ICE,TOTAL,ICE,ICE,True,True


### 4.4. ST_ACTIVIDADES_CORRIDO

In [84]:
# Crear df y validador
ST_ACTIVIDADES_CORRIDO, df_validador = generar_tablas_actividades_corrido(df=df_ied_ice_corrido_actividad_insumo, temp_parametros=TEMP_PARAMETROS_CORRIDO)

In [85]:
# Todo debe estar en true para no tener errores
df_validador

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T
0,IED,TOTAL,IED,IED,True,True
1,ICE,TOTAL,ICE,ICE,True,True
2,IED,TOTAL,IED,IED,True,True
3,ICE,TOTAL,ICE,ICE,True,True


In [86]:
# Cambio a nombres de versión original (asegura compatibilidad con el código anterior )
ST_ACTIVIDADES_CORRIDO = ST_ACTIVIDADES_CORRIDO.rename(columns={'DIFERENCIA_ABSOLUTA_T': 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL_T' : 'DIFERENCIA_PORCENTUAL'})

### 4.5. ST_PAISES_CERRADO_ACUMULADO

In [87]:
# IED

# Leer el archivo
df_ied_pais_acumulado = pd.read_excel(path_ied + ied_pais_file, sheet_name='AñoCerrado')

# Elegir columnas de interés (2019 - 2023)
df_ied_pais_acumulado = df_ied_pais_acumulado[['PAIS', 'Suma ult 5 años']]

# Cambiar nombres
df_ied_pais_acumulado = df_ied_pais_acumulado.rename(columns={'PAIS' : 'UNIDAD', 'Suma ult 5 años' : 'VALOR'})

# Columna de tipo
df_ied_pais_acumulado['TIPO'] = 'IED'

# Filtrar total
df_ied_pais_acumulado = df_ied_pais_acumulado[df_ied_pais_acumulado['UNIDAD'] != 'TOTAL']

# Ordenar el DataFrame por 'VALOR' de forma descendente
df_ied_pais_acumulado = df_ied_pais_acumulado.sort_values(by='VALOR', ascending=False).reset_index(drop=True)

# Asignar el ranking basado en el índice del DataFrame ordenado
df_ied_pais_acumulado['RANKING'] = df_ied_pais_acumulado.index + 1

In [88]:
# ICE

# Leer el archivo
df_ice_pais_acumulado = pd.read_excel(path_ied + ice_pais_file, sheet_name='AñoCerrado')

# Elegir columnas de interés (2019 - 2023)
df_ice_pais_acumulado = df_ice_pais_acumulado[['PAIS', 'Suma ult 5 años']]

# Cambiar nombres
df_ice_pais_acumulado = df_ice_pais_acumulado.rename(columns={'PAIS' : 'UNIDAD', 'Suma ult 5 años' : 'VALOR'})

# Columna de tipo
df_ice_pais_acumulado['TIPO'] = 'ICE'

# Filtrar total
df_ice_pais_acumulado = df_ice_pais_acumulado[df_ice_pais_acumulado['UNIDAD'] != 'TOTAL']

# Ordenar el DataFrame por 'VALOR' de forma descendente
df_ice_pais_acumulado = df_ice_pais_acumulado.sort_values(by='VALOR', ascending=False).reset_index(drop=True)

# Asignar el ranking basado en el índice del DataFrame ordenado
df_ice_pais_acumulado['RANKING'] = df_ice_pais_acumulado.index + 1

In [89]:
# Crear base agrupadada
ST_PAISES_CERRADO_ACUMULADO = pd.concat([df_ied_pais_acumulado, df_ice_pais_acumulado])

## 5. UNCTAD

In [90]:
# Definir la ruta completa del archivo
path = "C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/INVERSION/Bulk_UNCTAD/"
unctad_file = 'US_FdiFlowsStock.csv'
file_path = path + unctad_file

# Leer el archivo CSV con comas como separadores y puntos como decimales
df_unctad = pd.read_csv(file_path, sep=',', decimal='.', dtype= {
                                    'Year': str, 
                                    'Economy': str, 
                                    'Economy Label': str, 
                                    'Direction Label': str,
                                    'US$ at current prices in millions': float})

# Elegir flujos de inversión
df_unctad = df_unctad[df_unctad['Flow Label']=='Flow']

# Elegir columnas de interés
df_unctad = df_unctad[['Year', 'Economy', 'Economy Label', 'Direction Label', 'US$ at current prices in millions']]

# Reemplazar los valores en la columna 'Direction Label'
df_unctad['Direction Label'] = df_unctad['Direction Label'].replace({
    'Inward': 'IED',   # Inversión Extranjera Directa
    'Outward': 'ICE'   # Inversión de Colombia en el Exterior
})

# Reemplazar códigos de continentes por códigos UN SD
reemplazos = {
    '5100': '2', # Africa
    '5200': '19', # Americas
    '5300': '142', # Asia
    '5400': '150', # Europe
    '5500': '9' # Oceania
}
df_unctad['Economy'] = df_unctad['Economy'].replace(reemplazos)

# Cambio de nombres
df_unctad = df_unctad.rename(columns={
    'Year' : 'YEAR', 
    'Economy' : 'ECONOMY', 
    'Economy Label' : 'ECONOMY_LABEL', 
    'Direction Label' : 'DIRECTION_LABEL',
    'US$ at current prices in millions' : 'US_CURRENT_PRICES_MILLIONS'
})

# Filtrar años de interés
df_unctad = df_unctad[df_unctad['YEAR'].isin(['2018', '2019', '2020', '2021', '2022', '2023'])]

# Eliminar vacíos
df_unctad = df_unctad.dropna(subset=['US_CURRENT_PRICES_MILLIONS'])

# Convertir a número para eliminar ceros a la izquierda y luego de vuelta a cadena
df_unctad['ECONOMY'] = df_unctad['ECONOMY'].astype(int).astype(str)

## 6. Crear DF de validación

In [ ]:
# Colombia 
suma_total_ied_t_1_control_cerrado, suma_total_ied_t_control_cerrado
# Corrido
suma_total_ied_t_1_control_corrido, suma_total_ied_t_control_corrido

# ICE
# Cerrado
suma_total_ice_t_1_control_cerrado, suma_total_ice_t_control_cerrado
# Corrido
suma_total_ice_t_1_control_corrido, suma_total_ice_t_control_corrido

In [ ]:
# Colombia

df_control_colombia = pd.DataFrame({
    'AGRUPACION' : ['COLOMBIA'],
    'UNIDAD' : ['COLOMBIA'],
    'TIPO_PERIODO' : 'IED-CERRADO-COLOMBIA-PAISES',
    'SUMA_INVERSION_T_1' : suma_total_ied_t_1_control_cerrado,
    'SUMA_INVERSION_T': suma_total_ied_t_control_cerrado
})


df_control_colombia1 = pd.DataFrame({
    'AGRUPACION' : ['COLOMBIA'],
    'UNIDAD' : ['COLOMBIA'],
    'TIPO_PERIODO' : 'IED-CERRADO-COLOMBIA-ACTIVIDADES',
    'SUMA_INVERSION_T_1' : suma_total_ied_t_1_control_cerrado,
    'SUMA_INVERSION_T': suma_total_ied_t_control_cerrado
})

pd.concat([df_control_colombia, df_control_colombia1])

In [ ]:
# PAISES

# Obtener correlativa de países
df_paises = pd.DataFrame(session.sql("""
SELECT DISTINCT A.PAIS_INVERSION_BANREP, 
    A.PAIS_LLAVE_EXPORTACIONES
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
""").collect())

df_paises

In [ ]:
df_ied_ice_corrido_pais

In [ ]:
# Paises
# Cerrado
df_control_paises_ied_cerrado = df_ied_ice_corrido_pais.merge(df_paises, how='left', left_on='PAIS', right_on='PAIS_INVERSION_BANREP')
df_control_paises_ied_cerrado = df_control_paises_ied_cerrado[['YEAR', 'VALOR', 'TIPO', 'PAIS_LLAVE_EXPORTACIONES']]
df_control_paises_ied_cerrado['AGRUPACION'] = 'PAISES'
df_control_paises_ied_cerrado = df_control_paises_ied_cerrado.rename(columns={'PAIS_LLAVE_EXPORTACIONES' : 'UNIDAD'})

 # Filtro 
df_control_paises_ied_cerrado = df_control_paises_ied_cerrado[(df_control_paises_ied_cerrado['YEAR']=='2022') | (df_control_paises_ied_cerrado['YEAR']=='2023')]

# Pivot
pd.DataFrame(df_control_paises_ied_cerrado.pivot_table(
    index=['AGRUPACION', 'UNIDAD', 'TIPO'],
    columns='YEAR',
    values='VALOR',
    aggfunc='sum'
)).reset_index()

# Agregar tipo periodo
# Modificar tipo por tipo perido
if tipon == 'IED' entonces 'nsdfosdfjipsdf'

In [ ]:
# Corrido
df_ied_ice_pais_corrido_insumo

In [ ]:
# Paises
# Corrrido
df_control_paises_ied_corrido = df_ied_ice_pais_corrido_insumo.merge(df_paises, how='left', left_on='PAIS', right_on='PAIS_INVERSION_BANREP')
df_control_paises_ied_corrido = df_control_paises_ied_corrido[['YEAR', 'VALOR', 'PAIS_LLAVE_EXPORTACIONES']]
df_control_paises_ied_corrido['AGRUPACION'] = 'PAISES'
df_control_paises_ied_corrido = df_control_paises_ied_corrido.rename(columns={'PAIS_LLAVE_EXPORTACIONES' : 'UNIDAD'})

 # Filtro 
df_control_paises_ied_corrido = df_control_paises_ied_corrido[(df_control_paises_ied_corrido['YEAR']=='2022-2') | (df_control_paises_ied_corrido['YEAR']=='2023-2')]

# Definiri tipo periodo 

# Pivot
pd.DataFrame(df_control_paises_ied_corrido.pivot_table(
    index=['AGRUPACION', 'UNIDAD'],
    columns='YEAR',
    values='VALOR',
    aggfunc='sum'
)).reset_index()

# Que al pivto le sobreviva la columna tipo, en index, 

# Agregar tipo periodo


In [ ]:
# Continetnes
# Obtener correlativa de continentes
df_continentes = pd.DataFrame(session.sql("""
SELECT DISTINCT A.PAIS_INVERSION_BANREP, 
    A.REGION_NAME_EXPORTACIONES
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A;
""").collect())
df_continentes

In [ ]:
# cONTINENTES
# Cerrado
df_control_continentes_ied_cerrado = df_ied_ice_corrido_pais.merge(df_continentes, how='left', left_on='PAIS', right_on='PAIS_INVERSION_BANREP')
df_control_continentes_ied_cerrado = df_control_continentes_ied_cerrado[['YEAR', 'VALOR', 'TIPO', 'REGION_NAME_EXPORTACIONES']]
df_control_continentes_ied_cerrado['AGRUPACION'] = 'CONTINENTES'
df_control_continentes_ied_cerrado = df_control_continentes_ied_cerrado.rename(columns={'REGION_NAME_EXPORTACIONES' : 'UNIDAD'})

 # Filtro 
df_control_continentes_ied_cerrado = df_control_continentes_ied_cerrado[(df_control_continentes_ied_cerrado['YEAR']=='2022') | (df_control_continentes_ied_cerrado['YEAR']=='2023')]

# Pivot
pd.DataFrame(df_control_continentes_ied_cerrado.pivot_table(
    index=['AGRUPACION', 'UNIDAD', 'TIPO'],
    columns='YEAR',
    values='VALOR',
    aggfunc='sum'
)).reset_index()

In [ ]:
# Obtener correlativa de hubs
df_hubs = pd.DataFrame(session.sql("""
SELECT DISTINCT A.PAIS_INVERSION_BANREP, 
    A.HUB_NAME_EXPORTACIONES
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A;
""").collect())
df_hubs

In [ ]:
df_hubs['HUB_NAME_EXPORTACIONES'].unique()

In [ ]:
# Correlativa de TLCs
df_tlcs = pd.DataFrame(session.sql("""
SELECT DISTINCT A.PAIS_INVERSION_BANREP,       
    A.NOMBRE_TLC
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A;
""").collect())
df_tlcs

## 7. Subir a Snowflake

In [91]:
# Usar base de datos:
sql_database = """
USE DOCUMENTOS_COLOMBIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_database)

,status
0,Statement executed successfully.


In [92]:
# Usar esquema inversión:
sql_schema_inversion = """
USE SCHEMA INVERSION;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_inversion)

,status
0,Statement executed successfully.


In [93]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,INVERSION


In [94]:
# Lista de pd a subir
bases_de_datos = [
    ST_PAISES_CERRADO, 
    ST_PAISES_CORRIDO,
    ST_ACTIVIDADES_CERRADO,
    ST_ACTIVIDADES_CORRIDO,
    df_unctad,
    ST_PAISES_CERRADO_ACUMULADO
]

nombres_tablas = [
    'ST_PAISES_CERRADO', 
    'ST_PAISES_CORRIDO',
    'ST_ACTIVIDADES_CERRADO',
    'ST_ACTIVIDADES_CORRIDO',
    'UNCTAD', 
    'ST_PAISES_CERRADO_ACUMULADO'
]

In [95]:
# pd de verificación
df_resultados_verificacion = pd.DataFrame()
# Subir y verificar bases a Snowflake
for base, tabla in zip(bases_de_datos, nombres_tablas):
    # Cargar el DataFrame en Snowflake y capturar el mensaje de carga
    mensaje_carga = snow_func.snowflake_cargar_df(conn, base, f'{tabla}')
    
    # Verificar y almacenar el resultado en el DataFrame
    resultado = snow_func.snowflake_sql(conn, f"SELECT COUNT(*) FROM {tabla};")
    total_registros = resultado  # Extraer el total de registros

    # Crear un DataFrame temporal para la nueva fila
    nueva_fila = pd.DataFrame({
        'Tabla': [tabla],
        'Total_Registros': [total_registros],
        'Mensaje_Carga': [mensaje_carga]
    })

    # Concatenar la nueva fila al DataFrame de resultados
    df_resultados_verificacion = pd.concat([df_resultados_verificacion, nueva_fila], ignore_index=True)

In [96]:
# Ver resultados
df_resultados_verificacion

,Tabla,Total_Registros,Mensaje_Carga
0,ST_PAISES_CERRADO,COUNT(*) 0 95,DataFrame cargado exitosamente en la tabla: 95 filas en 1 chunks.\nTiempo de carga: 2.76 segundos.\nProceso terminado
1,ST_PAISES_CORRIDO,COUNT(*) 0 92,DataFrame cargado exitosamente en la tabla: 92 filas en 1 chunks.\nTiempo de carga: 1.85 segundos.\nProceso terminado
2,ST_ACTIVIDADES_CERRADO,COUNT(*) 0 22,DataFrame cargado exitosamente en la tabla: 22 filas en 1 chunks.\nTiempo de carga: 2.10 segundos.\nProceso terminado
3,ST_ACTIVIDADES_CORRIDO,COUNT(*) 0 22,DataFrame cargado exitosamente en la tabla: 22 filas en 1 chunks.\nTiempo de carga: 1.91 segundos.\nProceso terminado
4,UNCTAD,COUNT(*) 0 3048,DataFrame cargado exitosamente en la tabla: 3048 filas en 1 chunks.\nTiempo de carga: 1.92 segundos.\nProceso terminado
5,ST_PAISES_CERRADO_ACUMULADO,COUNT(*) 0 136,DataFrame cargado exitosamente en la tabla: 136 filas en 1 chunks.\nTiempo de carga: 1.84 segundos.\nProceso terminado


### Cerrar sesión, conexión y cursor

In [97]:
conn.close()
session.close()